## Week-2: Neural network training

#### What are the steps to train the neural network using tensorflow?
Like we did before with linear regression, we do the following 3 steps:

1- **Define model**: Specify how to compute output given input x, and paramters w,b. Answers the question what function are we using to calculate the output given x,w,b

In [ ]:
## defining model example in tensorflow
model = Sequential(
    [               
        tf.keras.Input(shape=(400,)),    #specify input size
        Dense(25, activation='sigmoid'), 
        Dense(15, activation='sigmoid'), 
        Dense(1,  activation='sigmoid') 
    ], name = "my_model" 
)    

2- **Specify loss and cost funcitions**: This should answer the question on how we determine how good the model is given predictions vs actual output

In [ ]:
## Specifying the loss function with optimizer in tensorflow
model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(0.001),
)

3- **Minimize the cross function**: When using a loss function the loss can be high, or can reduced. This step tries to reduce the loss calculated in the loss function set in the previous step using backpropogation. We used Gradient descent before, but tensorflow has a better alogrithm that can reduce cost.

In [ ]:
## Telling tensorflow to try to reduce the cost using backpropogation and by setting number of epochs(trials per dataset)
model.fit(
    X,y,
    epochs=20
)

#### How do you define models?
We define the model by a function that takes input x, and parameters w,b to predict an output y. The function used here is called activation function. There are multiple activation functions that we can choose from to fit our model: Sigmoid, ReLU, no activation function(linear), or softmax, which we will learn about later.

#### How to choose the activation function that will best fit my model?
**For the output layer**:
* Use sigmoid activation function if your prediction is a binary decision 1/0
* Use linear activation if your prediction is a number that can be positive or negative
* Use ReLU activation if your prediction is always a positive number

**For the hidden layers** Use ReLU instead of sigmoid as the ReLU is **faster to compute** as it just computes max unlike sigmoud that computes exponeniation and inverse, and because the ReLU function **is only flat in one part of the graph** unlike the sigmoid that is flat in two parts of the graph making it take more time to converge when trying to reduce the loss.

#### Why don't we just use linear functions instead of these activation functions?
If we start using a linear activation function in the hidden layers, the network won't be more any beneficiary than one a linear regression network as multiple linear functions will still lead to a linear activation function, hence the recommendation of using an activation function like ReLU in the hidden layers, to avoid the neetwork being multiple linear functions that is as good as a linear regression.

#### So far, logistic regression has been for a binary classification (1/0), how can we implement a model that predicts multi-class?
multi-class classification problems are usually refered to as softmax regression problems. Since we know that the all possible predictions add up to 1, softmax uses this to predict the correct classification of multiple classes. Softmax is a generalized model of the logisitic regression, since it can also include binary classification.
Softmax regression: \
![Softmax regression](/resources/C2_W2_SoftmaxReg_NN.png)

#### Cost function for the softmax regression:
Since softmax has multiple output possibilities, it will have more output units; leading to the following cost function: \
![Softmax cost](/resources/C2_W2_SoftMaxCost.png)

#### Can the softmax regression be improved?
Yes, the softmax can be improved to avoid numerical round-off errors; this approach will imporve the results of the softmax regression model, but will make the code less obvious.
We change the output layer to be linear, and the set from_logits to true, to avoid rounding the activation values, and use its function instead to avoid the rounding, to make the predictions more accurate.
![Improved implementation of softmax](/resources/C2_W2_softmax_accurate.png)

This means that the output of the model will be the sigmoid function not output, so we will need an extra step to use the sigmoid outputs to output posiblities of each class. See code below:


In [ ]:
preferred_model = Sequential(
    [ 
        Dense(25, activation = 'relu'),
        Dense(15, activation = 'relu'),
        Dense(4, activation = 'linear')   #<-- Note
    ]
)
preferred_model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),  #<-- Note
    optimizer=tf.keras.optimizers.Adam(0.001),
)

preferred_model.fit(
    X_train,y_train,
    epochs=10
)
p_preferred = preferred_model.predict(X_train)
sm_preferred = tf.nn.softmax(p_preferred).numpy()

## To select the most likely category, the softmax is not required. 
## One can find the index of the largest output using np.argmax().
for i in range(5):
    print( f"{p_preferred[i]}, category: {np.argmax(p_preferred[i])}")

#### SparseCategorialCrossentropy or CategoricalCrossEntropy
Tensorflow has two potential formats for target values and the selection of the loss defines which is expected.
- SparseCategorialCrossentropy: expects the target to be an integer corresponding to the index. For example, if there are 10 potential target values, y would be between 0 and 9. 
- CategoricalCrossEntropy: Expects the target value of an example to be one-hot encoded where the value at the target index is 1 while the other N-1 entries are zero. An example with 10 potential target values, where the target is 2 would be [0,0,1,0,0,0,0,0,0,0].


#### Multi-class vs Multi-label
Multi-class and Multi-label classification applications are two different things. The Multi-class will indentify a picture, for instance, whether it is a dog or a car, or other. While the multi-label classification application will can determine what objects a picture has, whether this picture has cars, pedestrians, and traffic lights, not only choosing one of them. \
![multi-label classification](/resources/multi-label%20classification.png)

#### ADAM optmization algorithm:
The ADAM optimization algorithm is an alogrithm that optimizes gradient descent by making smaller steps toward local minima bigger, and still avoids over stepping. It will have n+1 different learning rates (1 for each class +1 for b).
We can use the ADAM optimization in code by the following:

In [ ]:
model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(0.001), # <- optimizer with initial learning rate set to 0.001
)

#### Convolutional Neural Network:
Previously, we have been using Dense layers, where units in the hidden layer that looks at all outputs from previous layer. We sometimes can let these units look at specific outputs of the previous layer, to make the neural network faster, instead of each unit looking at all the previous outputs. This is called a Convolutional layer. It can be more practicle and efficient in some applications, and can make the model prone to over fitting! but again, this is only useful in certain applications, and cannot be used all the time.